In [1]:
%pip install --upgrade supabase==2.27.2 langchain-community==0.3.31 langchain-openai==0.3.35 --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\leandro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os

from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

C:\Users\leandro\AppData\Local\Temp\ipykernel_21944\2462536188.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-3-small')


In [4]:
import os
import pandas as pd  # Adicionado pandas para ler Excel
from langchain_core.documents import Document

SOURCE_DIRECTORY = "./books"
NUM_BOOKS = 17

def load_docs_from_excel():  # Renomeei para refletir a mudança
    langchain_documents = []
    
    for i in range(1, NUM_BOOKS + 1):
        book_id = f"{i:03d}"
        file_path = os.path.join(SOURCE_DIRECTORY, f"{book_id}.xlsx")  # Mudança para .xlsx

        try:
            # Lê o arquivo Excel. O header será interpretado automaticamente como as páginas
            df = pd.read_excel(file_path)
        except FileNotFoundError:
            print(f"Aviso: Arquivo {file_path} não encontrado. Pulando.")
            continue
        except Exception as e:
            print(f"Erro ao ler {file_path}: {e}")
            continue

        # df.columns contém os cabeçalhos (os números das páginas)
        for page in df.columns:
            # Pega os parágrafos daquela página (coluna) e remove valores vazios (NaN)
            paragraphs = df[page].dropna().tolist()

            for paragraph_text in paragraphs:
                # Garante que é string (caso venha algum número solto)
                text_str = str(paragraph_text).strip()

                if text_str and len(text_str) > 10:
                    metadata = {
                        "book_id": book_id, # Ex: 001, 002, ...
                        "page": int(page)
                    }
                    doc = Document(
                        page_content=text_str,
                        metadata=metadata
                    )
                    langchain_documents.append(doc)
    
    return langchain_documents

documents = load_docs_from_excel()
documents[-10:]

[Document(metadata={'book_id': '017', 'page': 310}, page_content='“Antes da guerra atual, dissemos que antecedendo a toda contenda armada se desencadeava uma luta mental, na qual intervinham legiões de pensamentos agrupados sob a égide de tendências opostas. Isto supõe, inquestionavelmente, o desenvolvimento de uma inusitada atividade mental nos diversos campos em que atua o pensamento, que corre de um ponto a outro em bandos ideológicos, ora com fins de exploração, ora para aninhar em outros climas mentais. O certo é que, após esse ir e vir, vão se gestando em muitas mentes humanas ideias cuja natureza obedece a origens sinistras. Temos assim explicado o incremento que tomaram na Europa os pensamentos de ódio de característica agressiva que, reagindo a toda conciliação com os pensamentos de outros povos, agitaram a tal ponto os espíritos e tornaram tão insuportável a atmosfera internacional que aqueles que pensaram dominar nessa luta mental optaram, ao não consegui-lo, por utilizar as

In [5]:
vector_store = SupabaseVectorStore.from_documents(
    documents,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=100,
)

In [6]:
import os
from supabase.client import create_client
from langchain_openai import OpenAIEmbeddings

# Configurações do Supabase
supabase_url = os.environ["SUPABASE_URL"]
supabase_key = os.environ["SUPABASE_SERVICE_KEY"]
supabase = create_client(supabase_url, supabase_key)

# Inicializa embeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

def similarity_search_with_score(query: str, k: int = 5):
    # Gera o embedding do texto da query
    query_embedding = embeddings.embed_query(query)

    # Chama a função RPC 'match_documents' no Supabase, passando o embedding e o número de resultados
    response = supabase.rpc(
        "match_documents",
        {
            "query_embedding": query_embedding,
            "match_count": k,
            'include_book_ids': None,
            'exclude_book_ids': None,  # pode passar filtros JSON se quiser
        }
    ).execute()

    results = response.data

    # Cada item em results tem id, content, metadata e similarity
    # Podemos montar uma lista de tuplas (Document, similarity)
    from langchain_core.documents import Document

    docs_with_scores = []
    for item in results:
        doc = Document(
            page_content=item["content"],
            metadata=item["metadata"]
        )
        score = item["similarity"]
        docs_with_scores.append((doc, score))

    return docs_with_scores

# Exemplo de uso
query = "me fale sobre a falta de vontade"
results = similarity_search_with_score(query, k=5)

for doc, score in results:
    print(f"Score: {score:.4f}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-----")


Score: 0.6231
Content: Deficiências e Propensões do Ser Humano A antideficiência que aconselhamos aplicar nos casosde falta de vontade é a decisão. Para que seja efetiva, teráde ser praticada conscientemente, com responsabilidade – como toda antideficiência exige –, sobrepondo-se comempenho à apatia até triunfar no forcejo psicológico. Oser deve demonstrar que é capaz de contrapor à abuliaque o domina a decisão de combatê-la. Conseguirá,assim, ter vontade para tudo.
Metadata: {'page': 41, 'book_id': '004'}
-----
Score: 0.5943
Content: Vontade é a força psíquica que move as energias hu manas e põe em atividade as determinações da inteligên ciapara o bem, defesa e superação do indivíduo. A falta devon tade anula essas possibilidades e prostra o ser naindifere n ça e na inércia, faz sua inteligência fracassar e chega até a perverter sua sensibilidade, porque o expõe atodas as tentações e contingências que o ameaçam.
Metadata: {'page': 38, 'book_id': '004'}
-----
Score: 0.5561
Content: A p